In [1]:
from IPython.display import Markdown, display  # Import
import warnings
import os
from dotenv import load_dotenv
import pandas as pd
from io import StringIO
import tiktoken

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Suppress any warnings for cleaner output
warnings.filterwarnings("ignore")

# Load environment variables from the .env file
load_dotenv()

# Set API keys from environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [11]:
def combined_table_markdown(detailed_business_requirements: str, permission_business_requirement: str, prohibitions_business_requirement: str) -> str:

    combined_table_prompt = '''
    # Instructions
    - Combine **Detailed_Business_Requirements**, **Permission_Business_Requirements**, and "Prohibitions_Business_Requirements** into a single list with three sections titled *Requirements*, *Permissions*, and *Prohibitions*
    - Ensure that output is in proper markdown

    # **Detailed_Business_Requirements**
    {detailed_business_requirements}

    # **Permission_Business_Requirements**
    {permission_business_requirements}

    # **Prohibitions_Business_Requirements**
    {prohibitions_business_requirements}

    '''

    # Create a chat prompt template using the detailed prompt.
    prompt = ChatPromptTemplate([
        ("system", combined_table_prompt),
    ])

    # Initialize the ChatOpenAI language model with a specific model name and temperature.
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

    # Combine the prompt, the language model, and the output parser into a processing chain.
    rag_chain = prompt | llm | StrOutputParser()


    # Asynchronously invoke the chain with the provided inputs.
    generation = rag_chain.invoke({
        "detailed_business_requirements": detailed_business_requirements,
        "permission_business_requirements": permission_business_requirement,
        "prohibitions_business_requirements": prohibitions_business_requirement,
    })

    return generation

In [9]:
df = pd.read_excel("../Data/Results/Query_Results.xlsx")
df

,Business_Requirements,Simplified_Business_Requirements,Detailed_Business_Requirements,Permission_Business_Requirements,Prohibitions_Business_Requirements
0,Applications pursuant to this section shall be...,"To comply with Regulation K, applications must...",### **Detailed Requirements and Actions for Re...,## **Permissions Based on Regulation K**\n\n- ...,## Prohibitions Based on Regulation K\n\n- **I...
1,If an application for permission to continue t...,The main business requirement for Regulation K...,### Detailed Requirements and Actions for Verb...,## **Permissions Based on Provided Context and...,- Conducting an activity without obtaining pri...
2,"United States banking organizations, with the ...",United States banking organizations can invest...,### **Detailed Requirements and Actions for Re...,## **Permissions for U.S. Banking Organization...,- United States banking organizations are proh...
3,Each full resolution plan shall include a stra...,The main business requirement for Regulation K...,### **Detailed Requirements and Actions for Ve...,## **Permissions Based on Provided Context and...,- The company is prohibited from failing to de...
4,A foreign bank may not establish a branch or a...,The main business requirement for Regulation K...,### **Detailed Requirements and Actions for Re...,## **Permissions Based on Regulation K**\n\n- ...,- Foreign banks are prohibited from establishi...
5,Each Edge or agreement corporation shall make ...,The main business requirements for Regulation ...,### **Detailed Requirements and Actions**\n\n#...,## **Permissions Based on Provided Context and...,- Corporations are prohibited from failing to ...


In [12]:
df = pd.read_excel("../Data/Results/Query_Results.xlsx")
generation = combined_table_markdown(detailed_business_requirements=df.loc[0, "Detailed_Business_Requirements"], permission_business_requirement=df.loc[0, "Permission_Business_Requirements"], prohibitions_business_requirement=df.loc[0, "Prohibitions_Business_Requirements"])
display(Markdown(generation))

# Business Requirements

## Requirements
1. **Filing Applications with the Federal Reserve Bank**
   - **Requirement**: All applications must be submitted to the appropriate Federal Reserve Bank.
   - **Action Steps**:
     - Identify the appropriate Federal Reserve Bank based on the applicant's location or business operations.
     - Prepare the application documents as per the guidelines provided by the Federal Reserve Bank.
     - Submit the application electronically or via mail, ensuring it reaches the designated office.

2. **Completeness of Application**
   - **Requirement**: An application is not considered complete until all requested information is provided and accepted.
   - **Action Steps**:
     - Review the application checklist provided by the Federal Reserve Bank to ensure all required information is included.
     - Gather necessary documentation such as financial statements, business plans, and any other required forms.
     - Conduct a preliminary review of the application to ensure all sections are filled out accurately.
     - Submit the application and await confirmation of its completeness from the Federal Reserve Bank.

3. **Acceptance of Application**
   - **Requirement**: The application must be accepted by the Federal Reserve Bank to proceed with the approval process.
   - **Action Steps**:
     - Monitor for communication from the Federal Reserve Bank regarding the acceptance status of the application.
     - Be prepared to respond to any requests for additional information or clarification promptly.
     - Keep records of all correspondence with the Federal Reserve Bank for future reference.

4. **Approval Conditions**
   - **Requirement**: Approval may be contingent upon the applicant's agreement to conduct activities under specific conditions or limitations.
   - **Action Steps**:
     - Review any conditions or limitations that may be imposed by the Federal Reserve Bank upon approval.
     - Prepare to negotiate or agree to these conditions as part of the approval process.
     - Develop an internal compliance plan to ensure adherence to any conditions set forth by the Federal Reserve Bank post-approval.
     - Document all agreements and conditions in a formal manner for future reference.

5. **Tracking Progress**
   - **Requirement**: Ensure that the progress of the application process is measurable and trackable.
   - **Action Steps**:
     - Create a timeline or project management tool to track the submission, acceptance, and approval stages of the application.
     - Set specific milestones for each stage of the application process (e.g., submission date, expected acceptance date, response to conditions).
     - Regularly update the tracking tool with any communications or changes in status from the Federal Reserve Bank.

6. **Time-bound Actions**
   - **Requirement**: Establish deadlines for each step in the application process.
   - **Action Steps**:
     - Set a deadline for the completion of the application preparation (e.g., 2 weeks from the start date).
     - Establish a timeline for submitting the application (e.g., within 1 week after preparation).
     - Create follow-up deadlines for checking the acceptance status (e.g., 1 week after submission).
     - Plan for a review period to address any conditions or limitations set by the Federal Reserve Bank (e.g., 1 month after acceptance).

## Permissions
- **Submission of Applications**
  - Permitted to file applications with the appropriate Federal Reserve Bank.
  - Allowed to submit applications electronically or via mail.

- **Completeness and Acceptance of Applications**
  - Permitted to provide all requested information to ensure the application is deemed complete.
  - Authorized to engage in communication with the Federal Reserve Bank to confirm acceptance of the application.

- **Approval Conditions**
  - Permitted to agree to conduct activities under specific conditions or limitations as set by the Federal Reserve Bank.
  - Allowed to negotiate or agree to conditions imposed upon approval.

- **Tracking and Monitoring**
  - Permitted to create and use a timeline or project management tool to track the application process.
  - Authorized to set specific milestones and deadlines for each stage of the application process.

- **Documentation and Compliance**
  - Permitted to document all agreements, conditions, and correspondence with the Federal Reserve Bank.
  - Allowed to develop an internal compliance plan to adhere to conditions set by the Federal Reserve Bank post-approval.

## Prohibitions
- **Incomplete Applications**: Submitting an application that does not contain all the requested information by the Federal Reserve Bank is prohibited.
- **Unaccepted Applications**: Proceeding with any activities without the Federal Reserve Bank's acceptance of the application is prohibited.
- **Non-compliance with Conditions**: Engaging in activities without agreeing to or adhering to specific conditions or limitations set by the Federal Reserve Bank upon approval is prohibited.